In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time


from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Base URL for company job listings
base_url = 'https://www.bain.com/careers/find-a-role/'

# TODO: Define the pull_company_name function
def pull_company_name(base_url, all_links, job_links):
    """
    Extracts job links from the company careers page.

    Parameters:
    base_url (str): The base URL of the company careers page.
    all_links (list): List of all links on the current page.
    job_links (list): List of collected job links.

    Returns:
    list: Updated list of job links.
    """

    for link in all_links:
        # TODO: Check if the link contains a certain pattern and is not already in job_links
        # Append the link to job_links if it meets the criteria
        
        href = link['href'] # extract href
        # check for 'jobid' in hrefs to find proper job_links
        # check for duplicates before appending to return value

        # check_url = base_url + href[href.find('position'):]
        target_url = base_url + href[href.find('position'):]
        if href and 'jobid' in href and target_url not in job_links:
            # parse target href and concat with base_url for proper url/link
            # append to job_links list 
            # FUNCTION IS CALLED 'pull_company_name' SO MAYBE ADD COMPANY NAME BEFORE LINK LATER? OR MAYBE I DID THIS WRONG AND THEY WANT A LIST OF CLICKABLE LINKS?? IS THAT POSSIBLE? ANYWAY COMEBACK LATER!
            job_links.append(target_url)

    # print('joblinks',job_links)        
    return job_links

# TODO: Define the button_company_name function
def button_company_name(driver, page, outer_loop_break, all_links):
    """
    Handles pagination by clicking the "next page"/"load more" button.

    Parameters:
    driver (WebDriver): The Selenium WebDriver instance.
    page (int): The current page number. (optional)
    outer_loop_break (bool): Flag to indicate when to stop scraping.
    all_links (list): List of all links on the current page.

    Returns:
    tuple: Updated page number and outer_loop_break flag.
    
    """
    # URL changes when clicking 'load more' but not like how i want...
    # POSSIBLE IDEA?? -> compare curr number of job links on page to total number of job links class="search__counter"
    counted_roles = driver.find_element(By.CLASS_NAME, 'search__counter').text
    #loop through string 'Showing: 1 - 10 of 121 roles' and extract int vals into list
        # make sure to change vals into integer int(i)
    num_vals = [int(i) for i in counted_roles.split() if i.isdigit()]
    # ([1, jobs_per_page, total_jobs])
    jobs_per_page = num_vals[1] 
    total_jobs = num_vals[-1]
    # print(jobs_per_page, total_jobs)


    try:
        # TODO: Locate the next page button and navigate to the next page if it exists

        # all_links param can be used to possibly check current page for links that fit criteria before moving onto next page
        #   but it doesn't seem all too necessary because clicking load more doesnt

        # loop through all links on current page specifically for job links 
            # add vals for total number of jobs on curr page
        job_link_count = sum('jobid' in link['href'] for link in all_links) 
        # print(job_link_count, jobs_per_page)
        
        # if total number of jobs on page == total number of jobs => BREAK LOOP & return
        if job_link_count == total_jobs:
            outer_loop_break = True
            return page, outer_loop_break
        
        # locate load_more button, wait 10 for page to load
        load_more = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.btn.btn--cta')))
        # print(load_more[0].text)

        # if load_more button exists,
        if load_more:
            # not 100% sure why yet, but load_more returns array -- need first elem of array
            load_more_buttons = load_more[0] # JS: grab first element of list of ALL 'load_more'
            # print("found button!")
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.btn.btn--cta'))) #check for button clickability
            driver.execute_script('arguments[0].scrollIntoView(true);', load_more_buttons) #scroll to load more button // went with JS method
            driver.execute_script('arguments[0].click();', load_more_buttons) # click load more button
            # wait for new content to load
            time.sleep(5) #5 might be kinda long but better to be safe
            
        else:
            # print('non found')
            # PROBLEM: why are we skipping this else ..., skips to excpetion instead.
                # SOLUTION: check for total num jobs == total num jobs on curr page
            outer_loop_break = True

    except Exception as e:
        print(f"Error occurred while trying to click the 'next page' button: {e}")
        outer_loop_break = True
    return page + 1, outer_loop_break

# WebDriver setup

options = Options()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.implicitly_wait(10)
driver.get(base_url)

# Initialize variables
job_links = []
prev_page = 0
page = 1

# Main scraping loop
while True:
    time.sleep(5)
    outer_loop_break = False
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    all_links = soup.find_all("a", href=True)

    # Scrape job links
    job_links = pull_company_name(base_url, all_links, job_links)

    # Handle pagination
    page, outer_loop_break = button_company_name(driver, page, outer_loop_break, all_links)

    if outer_loop_break:
        break

# Output the collected job links
print("Collected job links:", job_links)
# make sure im getting total num (121), no copies
# print(len(job_links))


# Close the WebDriver
driver.quit()


ModuleNotFoundError: No module named 'webdriver_manager'